In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
import sys

# 1️⃣ Chuyển working directory tới root project (nơi có thư mục src)
%cd /content/drive/MyDrive/chatbot

# 2️⃣ Thêm folder root vào sys.path để Python tìm src
sys.path.append(os.getcwd())

# 3️⃣ Kiểm tra
!ls
# Nên thấy thư mục src, app_gradio.py, notebooks, v.v.


/content/drive/MyDrive/chatbot
app_gradio.ipynb	 eval_result		 README.md
app_gradio_server.ipynb  LICENSE		 src
data			 model_evaluation.ipynb  train_models.ipynb
data_generating		 models
data_viz.ipynb		 rag_evaluation.ipynb


In [3]:
!pip install langchain langchain-community langchain-google-genai google-generativeai transformers torch sentence-transformers pinecone gradio peft bitsandbytes accelerate pymongo --quiet
!pip install pinecone --upgrade --quiet
!pip install -U bitsandbytes --quiet
!pip install rank-bm25 unidecode -q
!pip install pymongo rapidfuzz -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.9/146.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.9/745.9 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9

In [4]:
CUSTOM_CSS = """
body {
    margin: 0;
    background: radial-gradient(circle at top left, #eef2ff 0, #f5f7fb 45%, #ffffff 100%);
    font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", system-ui, sans-serif;
}

.gradio-container {
    padding: 0 !important;
}

/* App shell */
.app-shell {
    max-width: 1180px;
    margin: 24px auto 32px auto;
    border-radius: 18px;
    background: rgba(255,255,255,0.96);
    box-shadow: 0 20px 50px rgba(15,23,42,0.10);
    border: 1px solid #d1d5db;
    padding: 0;
    overflow: hidden;
}

/* Top bar */
.app-header-bar {
    display: flex;
    flex-direction: column;
    padding: 18px 24px 14px 24px;
    border-bottom: 1px solid #e5e7eb;
    background: linear-gradient(to right, #ffffff, #f9fafb);
}

.app-header-title {
    font-size: 42px;
    font-weight: 900;
    color: #111827;
    letter-spacing: -0.6px;
    margin-bottom: 6px;
    line-height: 1.1;
}

.app-header-subtitle {
    font-size: 14px;
    color: #6b7280;
}

/* Main layout = 2 columns */
.app-main-row {
    padding: 18px 24px 20px 24px;
    display: grid;
    grid-template-columns: 320px minmax(0, 1fr);
    column-gap: 18px;
    row-gap: 16px;
    align-items: stretch;
}

/* Responsive (mobile-ish) */
@media (max-width: 900px) {
    .app-main-row {
        grid-template-columns: minmax(0, 1fr);
    }
}

/* Sidebar */
.sidebar-panel {
    background: #f9fafb;
    padding: 14px 14px 16px 14px;
    border-radius: 14px;
    border: 1px solid #e5e7eb;
    display: flex;
    flex-direction: column;
    gap: 10px;
    height: 100%;
}

.section-title {
    font-size: 15px;
    font-weight: 700;
    color: #1f2937;
    margin-bottom: 8px;
}

/* Session status */
.current-session-status {
    font-size: 13px;
    margin-top: 6px;
    margin-bottom: 10px;
    padding: 10px;
    border-radius: 10px;
    background: #e0f2fe;
    color: #0c4a6e;
    font-weight: 600;
    border: 1px solid #bae6fd;
}

/* Chat panel */
.chat-panel {
    background: #ffffff;
    padding: 14px 16px 16px 16px;
    border-radius: 14px;
    border: 1px solid #e5e7eb;
    display: flex;
    flex-direction: column;
    gap: 10px;
    height: 100%;
}

/* Chatbot */
.gr-chatbot {
    background: #ffffff !important;
    border-radius: 12px !important;
    border: 1px solid #d1d5db !important;
}

/* Inputs */
textarea, input, select {
    font-size: 14px !important;
    border-radius: 10px !important;
    border-color: #d1d5db !important;
}

/* Buttons */
button {
    font-size: 14px !important;
    border-radius: 10px !important;
    padding: 9px 16px !important;
}

/* primary / secondary */
.btn-primary {
    background: #2563eb !important;
    color: #ffffff !important;
}
.btn-primary:hover {
    background: #1d4ed8 !important;
    box-shadow: 0 6px 15px rgba(37,99,235,0.25);
}

.btn-secondary {
    background: #f3f4f6 !important;
    color: #374151 !important;
    border-color: #d1d5db !important;
}
.btn-secondary:hover {
    background: #e5e7eb !important;
}

/* References summary */
#references-summary {
    margin-top: 4px;
    padding: 10px 12px;
    border-radius: 10px;
    background: #f9fafb;
    border: 1px solid #e5e7eb;
    font-size: 13px;
    color: #374151;
}
#references-summary b {
    color: #1f2937;
    font-size: 14px;
}

/* References detailed */
#references-full {
    max-height: 240px;
    overflow-y: auto;
    padding-right: 4px;
    font-size: 13px;
}

/* Accordion */
.gr-accordion {
    margin-top: 4px;
}

/* Examples */
.gr-examples {
    padding: 4px 0;
    margin-top: 6px;
    background: transparent !important;
    border: none !important;
}
.gr-examples > div {
    background: #f3f4f6 !important;
    border-radius: 8px !important;
    border: 1px solid #e5e7eb !important;
}

/* Footer */
.app-footer {
    padding: 6px 24px 10px 24px;
    border-top: 1px solid #e5e7eb;
    font-size: 11px;
    color: #9ca3af;
}
"""

In [5]:
import gradio as gr
from src.logic_module import (
    list_sessions,
    create_session_handler,
    load_session_handler,
    delete_session_handler,
    chat_fn,
)

# ================= Helper ================= #
def convert_to_messages(chat_history):
    """
    Chuyển chat_history về dạng list dict {"role":..., "content":...}
    Hỗ trợ:
        - list of tuples [(user, bot), ...]
        - list of dict [{"role":..., "content":...}, ...]
    """
    messages = []
    if not chat_history:
        return messages
    for item in chat_history:
        if isinstance(item, dict) and "role" in item and "content" in item:
            messages.append(item)
        elif isinstance(item, (list, tuple)):
            if len(item) >= 1:
                messages.append({"role": "user", "content": item[0]})
            if len(item) >= 2:
                messages.append({"role": "assistant", "content": item[1]})
    return messages

# ================= Gradio UI ================= #
with gr.Blocks() as demo:
    # CSS
    gr.Markdown(f"<style>{CUSTOM_CSS}</style>")

    # Global states
    state_session = gr.State(value=None)
    state_cache = gr.State(value=[])

    with gr.Column(elem_classes=["app-shell"]):
        # Header
        gr.Markdown("""
        <div class='app-header-bar' style="padding: 10px 0;">
            <div class='app-header-title' style="font-size: 42px; font-weight: 800; margin-bottom: 6px;">
                Vietnam Legal Assistance
            </div>
            <div class='app-header-subtitle' style="font-size: 22px; font-weight: 600; opacity: 0.9;">
                Preview interface for a legal chatbot focused on Vietnamese business and corporate law.
            </div>
        </div>
        """)

        # Main layout
        with gr.Row(elem_classes=["app-main-row"]):
            # ---------- Left sidebar ----------
            with gr.Column(scale=1):
                with gr.Group(elem_classes=["sidebar-panel"]):
                    gr.Markdown("<div class='section-title'>Sessions</div>")

                    new_session_name = gr.Textbox(label="New session name (optional)", lines=1)
                    new_session_btn = gr.Button("Create new session", elem_classes=["btn-primary"])

                    session_list = gr.Radio(
                        label="Saved sessions",
                        choices=[lbl for lbl, _ in list_sessions()],
                        interactive=True,
                    )

                    current_session_info = gr.Markdown(
                        "Current session: <em>none selected</em>",
                        elem_classes=["current-session-status"],
                    )

                    with gr.Row(equal_height=True):
                        load_btn = gr.Button("Load", elem_classes=["btn-secondary"])
                        delete_btn = gr.Button("Delete", elem_classes=["btn-secondary"])
                        refresh_btn = gr.Button("Refresh", elem_classes=["btn-secondary"])

            # ---------- Right chat ----------
            with gr.Column(scale=3):
                with gr.Group(elem_classes=["chat-panel"]):
                    gr.Markdown("<div class='section-title'>Conversation</div>")

                    chatbot = gr.Chatbot(height=360, type="messages", show_label=False)

                    refs_summary = gr.HTML(
                        value="<p>Legal references will appear here after you send a question.</p>",
                        elem_id="references-summary",
                    )

                    with gr.Accordion("All legal references (detailed)", open=False):
                        refs_full = gr.HTML(value="<p>No detailed references yet.</p>", elem_id="references-full")

                    msg = gr.Textbox(
                        label="Your question",
                        placeholder="For example: What are the legal conditions to set up an LLC in Vietnam?",
                        lines=1,
                    )

                    example_questions = [
                        "What are the conditions to establish a limited liability company in Vietnam?",
                        "Is there a minimum charter capital to open a company?",
                        "Can a foreigner open a company in Vietnam?",
                        "What key clauses should a labor contract include?",
                    ]

                    gr.Examples(examples=example_questions, inputs=msg, label="Suggested questions (click to use)")

                    with gr.Row(variant="compact"):
                        clear_btn = gr.Button("Clear conversation (local)", elem_classes=["btn-secondary"])
                        send = gr.Button("Send question", elem_classes=["btn-primary"])

        gr.Markdown("<div class='app-footer'>Preview UI only. Backend logic is provided by <code>src.logic_module</code>.</div>")

    # ================= Callbacks ================= #

    # ----- Create session -----
    def _create_session(name):
        sess, chat_history, cache, refs_html = create_session_handler(name)
        refs_sum = refs_html
        refs_det = refs_html
        label = name.strip() if name and name.strip() else str(sess)
        current_label = f"Current session: <b>{label}</b>"
        choices = [lbl for lbl, _ in list_sessions()]
        return (
            sess,
            convert_to_messages(chat_history),
            cache,
            refs_sum,
            refs_det,
            current_label,
            gr.update(choices=choices, value=label if label in choices else None),
        )

    new_session_btn.click(
        _create_session,
        inputs=[new_session_name],
        outputs=[state_session, chatbot, state_cache, refs_summary, refs_full, current_session_info, session_list],
    )

    # ----- Load session -----
    def _load_session(selected_label):
        sess, chat_history, cache, refs_html = load_session_handler(selected_label)
        refs_sum = refs_html
        refs_det = refs_html
        label = selected_label or "none selected"
        current_label = f"Current session: <b>{label}</b>"
        return sess, convert_to_messages(chat_history), cache, refs_sum, refs_det, current_label

    load_btn.click(
        _load_session,
        inputs=[session_list],
        outputs=[state_session, chatbot, state_cache, refs_summary, refs_full, current_session_info],
    )

    # ----- Delete session -----
    def _delete_session(selected_label):
        sess, chat_history, cache, refs_html = delete_session_handler(selected_label)
        refs_sum = refs_html
        refs_det = refs_html
        current_label = "Current session: <em>none selected</em>"
        choices = [lbl for lbl, _ in list_sessions()]
        return sess, convert_to_messages(chat_history), cache, refs_sum, refs_det, current_label, gr.update(choices=choices, value=None)

    delete_btn.click(
        _delete_session,
        inputs=[session_list],
        outputs=[state_session, chatbot, state_cache, refs_summary, refs_full, current_session_info, session_list],
    )

    # ----- Refresh sessions -----
    refresh_btn.click(lambda: gr.update(choices=[lbl for lbl, _ in list_sessions()]), inputs=None, outputs=session_list)

    # ----- Chat handler -----
    def _chat(sess, chat_history, user_msg, cache):
        sess2, chat2, cache2, refs_html = chat_fn(sess, chat_history, user_msg, cache)
        return sess2, convert_to_messages(chat2), cache2, refs_html, refs_html

    send.click(_chat, inputs=[state_session, chatbot, msg, state_cache], outputs=[state_session, chatbot, state_cache, refs_summary, refs_full])
    msg.submit(_chat, inputs=[state_session, chatbot, msg, state_cache], outputs=[state_session, chatbot, state_cache, refs_summary, refs_full])

    # ----- Clear local chat -----
    def _clear_local():
        return None, [], [], "<p>References cleared.</p>", "<p>No detailed references.</p>"

    clear_btn.click(_clear_local, inputs=None, outputs=[state_session, chatbot, state_cache, refs_summary, refs_full])

# ================= Launch ================= #
if __name__ == "__main__":
    demo.launch(server_name="127.0.0.1", debug=True)


/tmp/ipython-input-2632703577.py:84: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(height=360, type="messages", show_label=False)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8fad2abcda08b66870.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[⏱️ Processing time: 6.025 seconds]
Rewrite: cảm ơn bạn
[⏱️ Processing time: 6.729 seconds]
Rewrite: Bạn có thể cung cấp chi tiết về cách đăng ký thành lập doanh nghiệp không?
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8fad2abcda08b66870.gradio.live


# Del Session

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

uri = os.getenv("MONGO_URI")
db_name = os.getenv("MONGO_DB")
col_name = os.getenv("MONGO_COLLECTION")

from pymongo import MongoClient
result = MongoClient(uri)[db_name][col_name].delete_many({})
print("Đã xoá", result.deleted_count, "document.")


Đã xoá 10 document.
